In [11]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
findspark.init()

In [6]:
spark = SparkSession.builder.appName('Averages').getOrCreate()

24/05/06 09:16:46 WARN Utils: Your hostname, Bisheshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.163.225 instead (on interface en0)
24/05/06 09:16:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 09:16:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [38]:
df = spark.read.csv('./data/fc_transaction_base.csv',header='true', inferSchema='true')

In [42]:
df.printSchema()
df.show(5)

root
 |-- tran_date: date (nullable = true)
 |-- account_number: string (nullable = true)
 |-- branch: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- lcy_amount: double (nullable = true)
 |-- transaction_code: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- dc_indicator: string (nullable = true)
 |-- is_salary: integer (nullable = true)

+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+
| tran_date|     account_number|branch|product|lcy_amount|transaction_code|        description1|dc_indicator|is_salary|
+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+
|2020-06-09|02XYZXYZ10017529992|    15|    SBA|   14500.0|              CI|ATM WDL /00463701...|    withdraw|        0|
|2020-06-14|02XYZXYZ10017529992|    15|    SBA|   20000.0|              CI|ATM WDL /00463701...|    withdraw|        0|
|2020-06-01

In [41]:
df = df.withColumn("tran_date", to_date(col("tran_date"), "yyyy-MM-dd"))

In [43]:
df = df.withColumn("month", month(col("tran_date")))

In [55]:
df1 = df.fillna(0)

In [56]:
stats_df = df1.groupBy("account_number", "month").agg(
    avg("lcy_amount").alias("avg_lcy_amount"),
    stddev("lcy_amount").alias("stddev_lcy_amount"),
    variance("lcy_amount").alias("variance_lcy_amount")
)
stats_df=stats_df.fillna(0)

In [60]:
stats_df.show(5)

+--------------------+-----+------------------+-----------------+--------------------+
|      account_number|month|    avg_lcy_amount|stddev_lcy_amount| variance_lcy_amount|
+--------------------+-----+------------------+-----------------+--------------------+
| 02XYZXYZ10017500750|    6|           12677.3|9573.214008367304| 9.164642644999999E7|
| 02XYZXYZ10017541580|    6|           47578.0|              0.0|                 0.0|
|02XYZXYZ110017507489|    6|2897.8050000000003|5910.418552495097|3.4933047465678245E7|
| 02XYZXYZ10017544285|    6|          5775.335|6076.997550217542| 3.692989922535001E7|
| 02XYZXYZ10017557554|    6|           8166.29|              0.0|                 0.0|
+--------------------+-----+------------------+-----------------+--------------------+
only showing top 5 rows



In [61]:
stats1_df = stats_df.groupBy("account_number").agg(
    avg("avg_lcy_amount").alias("o_avg_lcy_amount"),
    avg("stddev_lcy_amount").alias("o_stddev_lcy_amount"),
    avg("variance_lcy_amount").alias("o_variance_lcy_amount")
)
stats1_df=stats1_df.fillna(0)

In [62]:
stats1_df.show(10)

+-------------------+------------------+-------------------+---------------------+
|     account_number|  o_avg_lcy_amount|o_stddev_lcy_amount|o_variance_lcy_amount|
+-------------------+------------------+-------------------+---------------------+
|02XYZXYZ10017544491|          5168.282| 2868.2281717247592|  2.468019853522667E7|
|02XYZXYZ10017529992|12374.278981481482| 13360.877610924514|  1.959612875635878E8|
|02XYZXYZ10017507450|13609.916130952382| 17873.931396962533| 3.4882907271013135E8|
|02XYZXYZ10017534270|11508.422777777778|  8886.244229831114|  8.974810524266389E7|
|02XYZXYZ10017272701| 85141.76018586352| 257036.99635041784| 1.654478943336487...|
|02XYZXYZ10017287701|16931.915641025644|  39975.12286751219| 2.3108435650068364E9|
|02XYZXYZ10017518608| 5652.844851364523|  8039.789681342617|  7.639431580287786E7|
|02XYZXYZ10017549354| 111887.2452785409|  264850.6287276143| 1.966134121744724E11|
|02XYZXYZ10017549474| 38520.43791005291|  70390.40730903004| 1.086996475061219...|
|02X

In [63]:
df = spark.read.csv('./data/fc_account_master.csv',header='true', inferSchema='true')

In [64]:
df.show(5)

+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
|     account_number|customer_code|product|product_category|acc_open_date|acc_closed_date|active_flag|
+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
|02XYZXYZ10015592101|    KL0255921|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015593701|    KL0255937|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015593801|    KL0255938|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015594801|    KL0255948|    SBA|           SBANU|   1999-03-22|           NULL|          0|
|02XYZXYZ10015597601|    KL0255976|    SBA|           SBPPS|   1999-04-22|           NULL|          0|
+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
only showing top 5 rows



In [67]:
stats2_df = stats1_df.join(df, on='account_number', how='inner')

In [69]:
stats3_df = stats2_df.select('account_number', 'customer_code','o_avg_lcy_amount','o_stddev_lcy_amount','o_variance_lcy_amount')

In [70]:
stats3_df=stats3_df.show(10)

+-------------------+-------------+------------------+-------------------+---------------------+
|     account_number|customer_code|  o_avg_lcy_amount|o_stddev_lcy_amount|o_variance_lcy_amount|
+-------------------+-------------+------------------+-------------------+---------------------+
|02XYZXYZ10017544491|    PB2143036|          5168.282| 2868.2281717247592|  2.468019853522667E7|
|02XYZXYZ10017529992|    PB2083592|12374.278981481482| 13360.877610924514|  1.959612875635878E8|
|02XYZXYZ10017507450|    KL2027115|13609.916130952382| 17873.931396962533| 3.4882907271013135E8|
|02XYZXYZ10017534270|    PB2097839|11508.422777777778|  8886.244229831114|  8.974810524266389E7|
|02XYZXYZ10017272701|    KL2009222| 85141.76018586352| 257036.99635041784| 1.654478943336487...|
|02XYZXYZ10017287701|    PB2075365|16931.915641025644|  39975.12286751219| 2.3108435650068364E9|
|02XYZXYZ10017518608|    KL2054409| 5652.844851364523|  8039.789681342617|  7.639431580287786E7|
|02XYZXYZ10017549354|    PB216